In [20]:
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
import numpy as np
import csv
errors = list()
k0=np.random.uniform(0,1)
k1=np.random.uniform(0,1)
def loadCsv(filename):
        trainSet = []
        
        lines = csv.reader(open(filename, 'r'))
        dataset = list(lines)
        for i in range(len(dataset)):
                for j in range(4):
                        #print("DATA {}".format(dataset[i]))
                        dataset[i][j] = float(dataset[i][j])
                trainSet.append(dataset[i])
        return trainSet


In [21]:
# print(k0,k1)

In [22]:
def minmax(dataset):
        minmax = list()
        stats = [[min(column), max(column)] for column in zip(*dataset)]
        return stats
 
# Rescale dataset columns to the range 0-1
def normalize(dataset, minmax):
        for row in dataset:
                for i in range(len(row)-1):
                        row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 
# Convert string column to float
def column_to_float(dataset, column):
        for row in dataset:
                try:
                        row[column] = float(row[column])
                except ValueError:
                        print("Error with row",column,":",row[column])
                        pass
 
# Convert string column to integer
def column_to_int(dataset, column):
        class_values = [row[column] for row in dataset]
        unique = set(class_values)
        lookup = dict()
        for i, value in enumerate(unique):
                lookup[value] = i
        for row in dataset:
                row[column] = lookup[row[column]]
        return lookup
 

In [23]:
def cross_validation_split(dataset, n_folds):
        dataset_split = list()
        dataset_copy = list(dataset)
        fold_size = int(len(dataset) / n_folds)
        for i in range(n_folds):
                fold = list()
                while len(fold) < fold_size:
                        index = randrange(len(dataset_copy))
                        fold.append(dataset_copy.pop(index))
                dataset_split.append(fold)
        return dataset_split
 
# Calculate accuracy percentage
def accuracy_met(actual, predicted):
        correct = 0
        for i in range(len(actual)):
                if actual[i] == predicted[i]:
                        correct += 1
        return correct / float(len(actual)) * 100.0
 

In [24]:
# Evaluate an algorithm using a cross validation split
def run_algorithm(dataset, algorithm, n_folds, *args):
        
        folds = cross_validation_split(dataset, n_folds)
        #for fold in folds:
                #print("Fold {} \n \n".format(fold))
        scores = list()
        for fold in folds:
                #print("Test Fold {} \n \n".format(fold))
                train_set = list(folds)
                train_set.remove(fold)
                train_set = sum(train_set, [])
                test_set = list()
                for row in fold:
                        row_copy = list(row)
                        test_set.append(row_copy)
                        row_copy[-1] = None
                predicted = algorithm(train_set, test_set, *args)
                actual = [row[-1] for row in fold]
                accuracy = accuracy_met(actual, predicted)
                cm = confusion_matrix(actual, predicted)
                print('\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in cm]))
                #confusionmatrix = np.matrix(cm)
                FP = cm.sum(axis=0) - np.diag(cm)
                FN = cm.sum(axis=1) - np.diag(cm)
                TP = np.diag(cm)
                TN = cm.sum() - (FP + FN + TP)
                print('False Positives\n {}'.format(FP))
                print('False Negetives\n {}'.format(FN))
                print('True Positives\n {}'.format(TP))
                print('True Negetives\n {}'.format(TN))
                TPR = TP/(TP+FN)
                print('Sensitivity \n {}'.format(TPR))
                TNR = TN/(TN+FP)
                print('Specificity \n {}'.format(TNR))
                Precision = TP/(TP+FP)
                print('Precision \n {}'.format(Precision))
                Recall = TP/(TP+FN)
                print('Recall \n {}'.format(Recall))
                Acc = (TP+TN)/(TP+TN+FP+FN)
                print('Áccuracy \n{}'.format(Acc))
                Fscore = 2*(Precision*Recall)/(Precision+Recall)
                print('FScore \n{}'.format(Fscore))
                k=cohen_kappa_score(actual, predicted)
                print('Çohen Kappa \n{}'.format(k))
                scores.append(accuracy)
        return scores
 

In [25]:
# Calculate neuron activation for an input
def activate(weights, inputs):
        
        activation = weights[-1]
        print('----',len(inputs))
        print('---->',len(weights))
        for i in range(len(weights)-1):
                activation += weights[i] * inputs[i]
        return activation
 
# Transfer neuron activation
def transfer(activation):
        return k0+k1*activation


In [26]:
def forward_propagate(network, row):
        inputs = row
        for layer in network:
                new_inputs = []
                for neuron in layer:
                        print('--------neuron',neuron)
                        print('----------neuron weights',neuron['weights'])
                        print('----------input',inputs)
                        activation = activate(neuron['weights'], inputs) 
                        print('-----activation',activation)
                        neuron['output'] = transfer(activation)
                        print('--------after activation',neuron['output'])
                        new_inputs.append(neuron['output'])
                inputs = new_inputs
        return inputs,activation

In [27]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
        return k1

In [28]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected,activation):
    global k0
    global k1
    global errors
    for i in reversed(range(len(network))):
            layer = network[i]
            
            k0_delta=list()
            k1_delta=list()
            if i != len(network)-1:
                    for j in range(len(layer)):
                            error = 0.0
                            for neuron in network[i + 1]:
                                    error += (neuron['weights'][j] * neuron['delta'])
                            errors.append(error)
                            #if i==(len(network)-1):
                             #   k1_delta.append(error)
            else:
                    for j in range(len(layer)):
                            neuron = layer[j]
                            errors.append(expected[j] - neuron['output'])
            for j in range(len(layer)):
                    neuron = layer[j]
                    neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
                    print('-----derivative',neuron['delta'])
                    if (i==len(network)-1):
                        print('-------i',i)
                        k0_delta.append(neuron['delta'])
                        k1_delta.append(activation * neuron['delta'])
                        print('----',k0_delta)
                        print('-----',k1_delta)


            sum_k0 = 0
            for t in k0_delta:
                sum_k0 = sum_k0 + t 
                avg_k0 = sum_k0/ len(k0_delta)
#                      try:
#                         sum_k0 = sum_k0 + t 
#                         avg_k0 = sum_k0/ len(k0_delta)
#                      except ZeroDivisionError:
#                           print("Variable x is not defined")
#                      except:
#                           print("Something else went wrong")




            sum_k1 = 0
            for t in k1_delta:
                sum_k1 = sum_k1 + t 
                avg_k1 = sum_k1 / len(k1_delta)
    l_rate=0.1
    k0=k0-l_rate*avg_k0
    k1=k1-l_rate*avg_k1
#                     try:
#                         sum_k1 = sum_k1 + t 
#                         avg_k1 = sum_k1 / len(k1_delta)

#                     except ZeroDivisionError:
#                           print("Variable x is not defined")
#                     except:
#   l_rate                        print("Something else went wrong")

In [29]:
# print(k0,k1)

In [30]:
# Backpropagate error and store in neurons


In [31]:
for i in reversed(range(3)):
                  print(i)

2
1
0


In [32]:
# Update network weights with error
def update_weights(network, row, l_rate):
        for i in range(len(network)):
                inputs = row[:-1]                
                if i != 0:
                        inputs = [neuron['output'] for neuron in network[i - 1]]
                for neuron in network[i]:
                        for j in range(len(inputs)):
                                temp = l_rate * neuron['delta'] * inputs[j] + mu * neuron['prev'][j]
                                
                                neuron['weights'][j] += temp
                                #print("neuron weight{} \n".format(neuron['weights'][j]))
                                neuron['prev'][j] = temp
                        temp = l_rate * neuron['delta'] + mu * neuron['prev'][-1]
                        neuron['weights'][-1] += temp
                        neuron['prev'][-1] = temp
              

In [33]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
  global k0
  global k1
  global errors
  for epoch in range(n_epoch):
          for row in train:
                  print('------row',row)
                  outputs,activation = forward_propagate(network, row)
                  #print(network)
                  expected = [0 for i in range(n_outputs)]
                  expected[row[-1]] = 1
                  #print("expected row{}\n".format(expected))
                  backward_propagate_error(network, expected,activation)
                  
                  update_weights(network, row, l_rate)
  print("Weights K0 = {k0} and k1 = {k1}".format(k0,k1))
  print("Training Error List {error}".format(error))

In [34]:
def initialize_network(n_inputs, n_hidden, n_outputs):
        network = list()

        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)], 'prev':[0 for i in range(n_inputs+1)]} for i in range(n_hidden)]        
        network.append(hidden_layer)
        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)], 'prev':[0 for i in range(n_inputs+1)]} for i in range(n_hidden)]        
        network.append(hidden_layer)
        output_layer = [{'weights':[random() for i in range(n_hidden + 1)],'prev':[0 for i in range(n_hidden+1)]} for i in range(n_outputs)]
        network.append(output_layer)
        print(network)
        #print(len(network))
        #print(output_layer)
        #print(hidden_layer)
        print('------------len of network',len(network)) 
        return network

In [35]:
#print(initialize_network(n_inputs=3, n_hidden=1, n_outputs=6))

In [36]:
# Make a prediction with a network
def predict(network, row):
        outputs = forward_propagate(network, row)
        return outputs.index(max(outputs))

In [37]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
        #print('-----',len(train))
        #for row in train:
            #print('len of row',row)
        n_inputs = len(train[0]) - 1
        n_outputs = len(set([row[-1] for row in train]))
        network = initialize_network(n_inputs, n_hidden, n_outputs)
        train_network(network, train, l_rate, n_epoch, n_outputs)
        #print("network {}\n".format(network))
        predictions = list()
        for row in test:
                prediction = predict(network, row)
                predictions.append(prediction)
        return(predictions)

In [38]:
# Test Backprop on Seeds dataset

seed(1)
# load and prepare data
filename = '/content/data.csv'
dataset = loadCsv(filename)
for i in range(len(dataset[0])-1):
        column_to_float(dataset, i)
# convert class column to integers
column_to_int(dataset, len(dataset[0])-1)
# normalize input variables
minmax = minmax(dataset)
normalize(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.1
mu=0.001
n_epoch = 1500
n_hidden = 4
scores = run_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)

print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Streaming output truncated to the last 5000 lines.
---- 4
----> 4
-----activation 0.06061750837240959
--------after activation 1.351360012608551
--------neuron {'weights': [0.36154981836769984, 0.09761779969395472, -0.31775972398373564, -0.33094806115040004], 'prev': [0.0, 0.0, 0.0, 0.0], 'output': 1.351360012608551, 'delta': -0.0}
----------neuron weights [0.36154981836769984, 0.09761779969395472, -0.31775972398373564, -0.33094806115040004]
----------input [0.07692307692307693, 0.7837837837837838, 0.6428571428571429, 2]
---- 4
----> 4
-----activation -0.43089939652706033
--------after activation 1.351360012608551
--------neuron {'weights': [0.794659664582562, 0.40904934445760044, 0.48704951208469033, 0.10445067120486441], 'prev': [0.0, 0.0, 0.0, 0.0], 'output': 1.351360012608551, 'delta': -0.0}
----------neuron weights [0.794659664582562, 0.40904934445760044, 0.48704951208469033, 0.10445067120486441]
----------input [0.07692307692307693, 0.7837837837837838, 0.6428571428571429, 2]
----

KeyboardInterrupt: ignored

In [ ]:
len(train[0])